In [1]:
"import sys
sys.path.append('..')
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Figur EINMAL erstellen
fig = go.FigureWidget()

# Slider für 3D Vektor
x_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='X:')
y_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Y:')
z_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Z:')

# Checkbox für Visualisierungsoptionen
show_grid = widgets.Checkbox(value=False, description='Gitter anzeigen')
show_grid_numbers = widgets.Checkbox(value=False, description='Gitter-Beschriftung')
show_axis_numbers = widgets.Checkbox(value=True, description='Achsen-Beschriftung')
show_second_vector = widgets.Checkbox(value=False, description='Zweiten Vektor anzeigen')

# Slider für zweiten Vektor (anfangs versteckt)
x2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='X2:')
y2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Y2:')
z2_slider = widgets.FloatSlider(value=1, min=-5, max=5, step=0.1, description='Z2:')

# Vektor-Info Widget
vector_info = widgets.HTML(
    value='<div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">'
          '<h3>Vektor Information:</h3>'
          '<p>Initialisierung...</p></div>'
)

def update_3d_plot(*args):
    # Vektoren berechnen
    v1 = [x_slider.value, y_slider.value, z_slider.value]
    v2 = [x2_slider.value, y2_slider.value, z2_slider.value]
    v_sum = [v1[0] + v2[0], v1[1] + v2[1], v1[2] + v2[2]]
    
    with fig.batch_update():
        fig.data = []  # Reset data
        
        # Koordinatenachsen immer anzeigen
        axis_length = 5
        fig.add_scatter3d(x=[-axis_length, axis_length], y=[0, 0], z=[0, 0],
                         mode='lines', line=dict(color='red', width=2),
                         name='X-Achse')
        fig.add_scatter3d(x=[0, 0], y=[-axis_length, axis_length], z=[0, 0],
                         mode='lines', line=dict(color='green', width=2),
                         name='Y-Achse')
        fig.add_scatter3d(x=[0, 0], y=[0, 0], z=[-axis_length, axis_length],
                         mode='lines', line=dict(color='blue', width=2),
                         name='Z-Achse')
        
        # Erster Vektor
        fig.add_cone(x=[v1[0]], y=[v1[1]], z=[v1[2]], u=[v1[0]], v=[v1[1]], w=[v1[2]],
                    sizeref=0.2, showscale=False, colorscale=[[0, 'blue'], [1, 'blue']])
        fig.add_scatter3d(x=[0, v1[0]], y=[0, v1[1]], z=[0, v1[2]],
                         mode='lines', line=dict(color='blue', width=4),
                         name=f'Vektor 1 ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})')
        
        # Zweiter Vektor und Summenvektor wenn aktiviert
        if show_second_vector.value:
            fig.add_cone(x=[v2[0]], y=[v2[1]], z=[v2[2]], u=[v2[0]], v=[v2[1]], w=[v2[2]],
                        sizeref=0.2, showscale=False, colorscale=[[0, 'red'], [1, 'red']])
            fig.add_scatter3d(x=[0, v2[0]], y=[0, v2[1]], z=[0, v2[2]],
                            mode='lines', line=dict(color='red', width=4),
                            name=f'Vektor 2 ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})')
            
            # Summenvektor
            fig.add_scatter3d(x=[0, v_sum[0]], y=[0, v_sum[1]], z=[0, v_sum[2]],
                            mode='lines', line=dict(color='green', width=4, dash='dot'),
                            name=f'Summe ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})')
        
        # Achsenbeschriftungen wenn aktiviert
        if show_axis_numbers.value:
            for i in range(-5, 6):
                if i != 0:
                    fig.add_scatter3d(x=[i], y=[0], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    fig.add_scatter3d(x=[0], y=[i], z=[0], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
                    fig.add_scatter3d(x=[0], y=[0], z=[i], mode='text',
                                    text=[str(i)], textposition='bottom center',
                                    showlegend=False)
        
        # Layout aktualisieren
        camera = fig.layout.scene.camera
        fig.update_layout(
            scene=dict(
                xaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                yaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                zaxis=dict(range=[-5, 5], showgrid=show_grid.value,
                          showbackground=False, gridcolor='lightgray'),
                aspectmode='cube',
                camera=camera
            ),
            width=800,
            height=800
        )
        
        # Vektor-Info aktualisieren
        info_html = f'''
        <div style="background-color: #f0f0f0; padding: 10px; border-radius: 5px;">
            <h3>Vektor Information:</h3>
            <p>Vektor 1: ({v1[0]:.2f}, {v1[1]:.2f}, {v1[2]:.2f})</p>
            <p>Berechnung der Länge (Pythagoras im 3D-Raum):</p>
            <p>√({v1[0]:.2f}² + {v1[1]:.2f}² + {v1[2]:.2f}²)</p>
            <p>√({v1[0]**2:.2f} + {v1[1]**2:.2f} + {v1[2]**2:.2f})</p>
            <p>√{v1[0]**2 + v1[1]**2 + v1[2]**2:.2f} = {np.sqrt(v1[0]**2 + v1[1]**2 + v1[2]**2):.2f}</p>
            {f"<p>Vektor 2: ({v2[0]:.2f}, {v2[1]:.2f}, {v2[2]:.2f})</p>" if show_second_vector.value else ""}
            {f"<p>Summenvektor: ({v_sum[0]:.2f}, {v_sum[1]:.2f}, {v_sum[2]:.2f})</p>" if show_second_vector.value else ""}
        </div>
        '''
        vector_info.value = info_html

# Alle Widgets mit Update-Funktion verbinden
for w in [x_slider, y_slider, z_slider, x2_slider, y2_slider, z2_slider,
          show_grid, show_grid_numbers, show_axis_numbers, show_second_vector]:
    w.observe(update_3d_plot, names='value')

# Layout erstellen
vector2_box = widgets.VBox([x2_slider, y2_slider, z2_slider])
vector2_box.layout.visibility = 'hidden'

def on_show_second_vector_change(change):
    vector2_box.layout.visibility = 'visible' if change.new else 'hidden'

show_second_vector.observe(on_show_second_vector_change, names='value')

# Checkboxen vertikal anordnen
checkbox_container = widgets.VBox([
    widgets.HBox([show_grid]),
    widgets.HBox([show_grid_numbers]),
    widgets.HBox([show_axis_numbers]),
    widgets.HBox([show_second_vector])
])

container = widgets.VBox([
    widgets.HBox([x_slider, y_slider, z_slider]),
    vector2_box,
    checkbox_container,
    fig,
    vector_info
])
display(container)

# Initiale Darstellung
update_3d_plot()

SyntaxError: unterminated string literal (detected at line 1) (2820330102.py, line 1)